In [3]:
# 1. K-NN 을 이용하여 breast cancer 데이터셋 분석하기​

############# 1-1) scikit-learn 내장 데이터 셋 불러오기 #############​
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()

# X (독립변수)와 y(종속변수)로 선언
X = cancer.data
y = cancer.target



"""
# 1-2) 직접 데이터셋 생성​
data = pd.DataFrame({"no_insects":[True, True, True, False, True, True, True, True, True, False],
                     "no_wilting": [True, True, True, True, True, True, False, True, True, True],
                     "no_diseases":[True,True,False,True,True,True,False,False,True,True],
                     "tree_health":["Good", "Good", "Poor", "Good", "Good", "Good", "Poor", "Poor", "Good", "Poor"]},
                    columns=["no_insects","no_wilting","no_diseases","tree_health"])

# 1-3) csv 파일 불러오기​
df = pd.read_csv("/...csv")
"""

############# 2) Data frame으로 데이터 확인​ #############
import pandas as pd
breast_cancer_df = pd.DataFrame(data=X, columns=cancer.feature_names)
breast_cancer_df['label']=y   # 맨끝쪽에 정답 label 추가
breast_cancer_df.head()       # 데이터의 상위 5개의 행 출력


############# 3) train/test/validation set 분리​ #############
# training / test / validation 데이터셋 나누기 (8:1:1)
from sklearn.model_selection import train_test_split

# train:test = 8:2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state =42, stratify=y)

# test data를 1:1 = test: validation
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size =0.5, random_state=42, stratify=y_test)

"""
Train_test_split의 파라미터 설명​

input data:  X (=cancer.data, 종속 변수)​
Output data: y (=cancer.target ,독립변수)​

Test_size = 0.2
train test 비율을 8:2로 설정​
test set을 다시 1:1로 validation set(검증데이터)과 test set으로 나눔​

Stratify:
y(=cancer.target) , stratify값을 target으로 지정해주면
각 클래스를 같은 비율로 train과 test에 할당​
즉, 한쪽에 치우치지 않도록​

Random_state:
일정한 값을 주어, 여러 번 실행되어도 같은 데이터를 추출​

​"""

############# 4) Normalization​ #############
# 데이터 전처리 (pre-processing)
# Standard scaler를 이용해, 모든 feature들이 평균 0, 분산 1인 정규분포를 갖도록 함​
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
std.fit(X_train)
X_train_scaled = std.transform(X_train)
X_val_scaled = std.transform(X_val)
X_test_scaled = std.transform(X_test)


############# 5) 모델 생성 및 학습​ #############
from sklearn.neighbors import KNeighborsClassifier

# KNN의 하이퍼파라미터 값 n_neighbors를 1로 설정​
knn = KNeighborsClassifier(n_neighbors=1)   # n_neighbors: 이웃의 개수

# KNN training
knn.fit(X_train_scaled, y_train)


############# 6) Evaluation​ #############
# 예측 정확도 출력
from sklearn import metrics

acc = knn.score(X_test_scaled, y_test)
print("Acc : {:.4f}".format(acc))


############# 7) 최적의 k 찾기​ #############
# validation set에서 정확도가 높은 이웃의 개수(k) 찾기
# validation set을 통해 최적의 k 찾기​

k_range = range(1,50)   # 1~49의 범위 지정
scores_list = []        # 리스트 생성

# 1-50까지 k에 따른 모델의 정확도를 반복문을 통해서 확인​
for k in k_range:
  knn = KNeighborsClassifier(n_neighbors = k)
  knn.fit(X_train_scaled, y_train)    # 모델 training
  y_pred = knn.predict(X_val_scaled)  # test data 예측
  scores_list.append(metrics.accuracy_score(y_val, y_pred))
  # Scores_list.append를 통해서 scores_list에 1-50까지의 k에 대한 정확도를 리스트에 저장​

############# 8) 최적의 k 찾기​ #############
# scores_list의 max값을 best_acc 변수에 저장    ​
# 1-50까지의 k 값에 대해 validation accuracy가 가장 높은 값이 best_acc에 저장된다​
best_acc = max(scores_list)

# Best_acc 값에 대응하는 index에 1을 더한 값을 best_k에 저장​
# 리스트의 index는 0부터 시작하므로, 1을 더해주어 최적의 k값을 구한다​
best_k = scores_list.index(best_acc)+1
print("best K: ", best_k)
print("prediction validation acc: {:.4f}".format(best_acc))


############# 9) 최적의 k로 재학습​​ #############
# 최적의 k로 test (Validation set을 통해 찾은 best_k값을 이용하여 knn학습​)
knn = KNeighborsClassifier(n_neighbors=best_k)

# KNN training
knn.fit(X_train_scaled, y_train)

# 예측 정확도 출력
acc = knn.score(X_test_scaled, y_test)
print("Acc: {:.4f}".format(acc))

Acc : 0.9123
best K:  3
prediction validation acc: 1.0000
Acc: 0.9649


In [5]:
# 1. Weighted KNN 을 이용하여 breast cancer 데이터셋 분석하기​

############# 1-1) scikit-learn 내장 데이터 셋 불러오기 #############​
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()

# X (독립변수)와 y(종속변수)로 선언
X = cancer.data
y = cancer.target


############# 2) Data frame으로 데이터 확인​ #############
import pandas as pd
breast_cancer_df = pd.DataFrame(data=X, columns=cancer.feature_names)
breast_cancer_df['label']=y   # 맨끝쪽에 정답 label 추가
breast_cancer_df.head()       # 데이터의 상위 5개의 행 출력


############# 3) train/test/validation set 분리​ #############
# training / test / validation 데이터셋 나누기 (8:1:1)
from sklearn.model_selection import train_test_split

# train:test = 8:2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state =42, stratify=y)

# test data를 1:1 = test: validation
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size =0.5, random_state=42, stratify=y_test)


############# 4) Normalization​ #############
# 데이터 전처리 (pre-processing)
# Standard scaler를 이용해, 모든 feature들이 평균 0, 분산 1인 정규분포를 갖도록 함​
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
std.fit(X_train)
X_train_scaled = std.transform(X_train)
X_val_scaled = std.transform(X_val)
X_test_scaled = std.transform(X_test)


############# 5) 모델 생성 및 학습​ #############
from sklearn.neighbors import KNeighborsClassifier

# KNN의 하이퍼파라미터 값 n_neighbors를 1로 설정​
knn = KNeighborsClassifier(n_neighbors=1)   # n_neighbors: 이웃의 개수

# KNN training
knn.fit(X_train_scaled, y_train)


############# 6) Evaluation​ #############
# 예측 정확도 출력
from sklearn import metrics

acc = knn.score(X_test_scaled, y_test)
print("Acc : {:.4f}".format(acc))


############# 7) 최적의 k 찾기​ #############
# validation set에서 정확도가 높은 이웃의 개수(k) 찾기
# validation set을 통해 최적의 k 찾기​

k_range = range(1,50)   # 1~49의 범위 지정
scores_list = []        # 리스트 생성

for k in k_range:
  knn = KNeighborsClassifier(n_neighbors=k, weights="distance")
  knn.fit(X_train_scaled, y_train)    # 모델 training
  y_pred = knn.predict(X_val_scaled)  # test data 예측
  scores_list.append(metrics.accuracy_score(y_val, y_pred))

# KNeighborsClassier의 파라미터 : weights ​
# 이웃을 설정할 때 가중치를 주는 방법, ​
# “distance”로 설정하면 거리가 가까운 이웃의 영향을 더 많이 받도록 설정 하는 것 ​


############# 8) 최적의 k, 최적의 k의 validation accuracy 출력​​ #############
best_acc = max(scores_list)
best_k = scores_list.index(best_acc)+1
print("best K: ", best_k)
print("prediction validation acc: {:.4f}".format(best_acc))

############# 9) 최적의 k로 weighted knn training​​​ #############
knn = KNeighborsClassifier(n_neighbors=best_k, weights="distance")
knn.fit(X_train_scaled, y_train)

############# 10) 예측 정확도 출력​​​​ #############
from sklearn import metrics

acc = knn.score(X_test_scaled, y_test)
print("Acc: {:.4f}".format(acc))


Acc : 0.9123
best K:  3
prediction validation acc: 1.0000
Acc: 0.9649
